In [89]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [90]:
df = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
df = df.drop(columns=['timestamp'])

df['rating'] = df['rating'] - 1

num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()

In [91]:
class MLDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_id'].values, dtype = torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype = torch.long)
        self.items = torch.tensor(df['item_id'].values, dtype = torch.long)
    def __len__(self):
        return len(self.ratings)
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]
train_dataset = MLDataset(df)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

In [92]:
class MLDataNN(nn.Module):
    def __init__(self):
        super(MLDataNN, self).__init__()
        self.user_embedding = nn.Embedding(num_users + 1, 50)
        self.item_embedding = nn.Embedding(num_items + 1, 50)
        self.fc_layers = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )
    def forward(self, user, item):
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)
        x = torch.cat([user_embed, item_embed], dim = 1)
        x = self.fc_layers(x)
        return x
model = MLDataNN()